# Venn diagramm

In [ ]:
install.packages('VennDiagram')
library(VennDiagram)

In [ ]:
plot_Venn_diagramm <- function(a, b, name_a, name_b) {
    options(repr.plot.res = 100) # set a high-definition resolution for the jupyter notebooks plots (DPI)
    notebook_plot_size(2, 2)
    
    return (draw.pairwise.venn(
    length(a),
    length(b),
    sum(a %in% b),
    category = c(name_a, name_b),
    fill = c("blue", "red"),
    alpha = c(0.5, 0.5),
    cat.pos = c(0, 0),
    scaled = TRUE))
}

plot_Venn_diagramm(oncokb_genes, candl_genes, "oncokb", "candl")

In [ ]:
notebook_plot_size(6, 6)
draw.triple.venn(
length(oncokb_genes),
length(cgi_genes),
length(civic_genes),
sum(oncokb_genes %in% cgi_genes),
sum(cgi_genes %in% civic_genes),
sum(civic_genes %in% oncokb_genes),
sum(cgi_genes %in% oncokb_genes & cgi_genes %in% civic_genes),
category = c("oncokb", "cgi", "civic"),
lty = rep("blank", 3),
fill = c("light blue", "pink", "green"),
alpha = rep(0.5, 3),
cat.pos = rep(0, 3),
cat.dist = rep(0.025, 3),
scaled = TRUE)

# ggballoonplot

In [ ]:
install.packages("ggpubr")
library(ggpubr)
theme_set(theme_pubr())

impact <- read.table("../../../data/cleaned_IMPACT_mutations_180508.txt", sep = "\t", stringsAsFactors = FALSE, header = TRUE)

ggballoonplot(as.data.frame(table(impact$Hugo_Symbol,
                                  impact$Variant_Type)),
              fill = "value") + scale_fill_viridis_c(option = "C")

In [ ]:
# install.packages("ggpubr")
# library(ggpubr)
# theme_set(theme_pubr())

colnames = c("is_in_oncokb", "is_in_civic", "is_in_cgi", "is_in_candl")

r <- data.frame()

for (col in colnames) {
    r <- rbind(r, impact %>% filter(!! sym(col) == "yes") %>%
                summarise(colnames = substr(col, 7, nchar(col)),
                          oncokb = sum(is_in_oncokb == "yes"),
                          civic  = sum(is_in_civic == "yes"),
                          cgi    = sum(is_in_cgi == "yes"),
                          candl  = sum(is_in_candl == "yes")))
}
r
ggballoonplot(r[,c("oncokb", "civic", "cgi", "candl")], fill = "value") + scale_fill_viridis_c(option = "C")

In [22]:
source("../src/utils/custom_tools.R")
setup_environment("../src/utils")

In [3]:
impact <- read.table("../data/cleaned_IMPACT_mutations_180508.txt", sep = "\t", stringsAsFactors = FALSE, header = TRUE)
#impact <- add_features("../data/", impact, annotations = TRUE)

# mutation distances checker

In [ ]:
check_distance <- function(starts, ends, mut_key, result) {
    if (length(starts) <= 1)
        return (result)
    
    for (i in 2:length(starts)) {
        if (starts[[i]] <= starts[[1]] & ends[[i]] >= starts[[1]] | starts[[i]] >= starts[[1]] & starts[[i]] <= ends[[1]]) {
            r1 <- data.frame(paste(starts[[1]], ends[[1]], sep = '_'), mut_key[[1]])
            r2 <- data.frame(paste(starts[[i]], ends[[i]], sep = '_'), mut_key[[i]])
            colnames(r1) <- c("v1", "v2")
            colnames(r2) <- c("v1", "v2")
            
            
            result <- rbind(result, r1)
            result <- rbind(result, r2)
            
            starts <- starts[-1]
            starts <- starts[-i]
            
            ends <- ends[-1]
            ends <- ends[-i]
            
            mut_key <- mut_key[-1]
            mut_key <- mut_key[-i]
            
            break
        }
    }
    
    if (length(starts) <= 1)
        return (result)
    
    return (check_distance(starts[2:length(starts)], ends[2:length(starts)], mut_key[2:length(starts)], result))
}

get_pos <- function(result) {
    if (nrow(result) == 0)
        return ("")
    else
        return (toString(result$v1))
}

get_mut <- function(result) {
    if (nrow(result) == 0)
        return ("")
    else
        return (toString(result$v2))
}




dd2 <- dd %>% group_by(Tumor_Sample_Barcode, Hugo_Symbol) %>% summarise(count = n(),
                                                                       pos = get_pos(check_distance(Start_Position, End_Position, mut_key, data.frame())),
                                                                       mut_key = get_mut(check_distance(Start_Position, End_Position, mut_key, data.frame()))) %>%
                                                             filter(pos != "")
nrow(dd2)
head(dd2, 10)

# Test cbioportal

In [32]:
utf8::utf8_print('\U0001f6a7 work in progress \U0001f6a7')

[1] "🚧​ work in progress 🚧​"


```shell
$ curl "http://www.cbioportal.org/webservice.do?cmd=getClinicalData&case_set_id=msk_impact_2017_all" --output cbioportal.txt```

In [21]:
cbioportal <- read.table("../../../data/other_databases/cbioportal.txt",
                         sep = "\t", stringsAsFactors = FALSE, header = TRUE, fill = TRUE)

In [22]:
head(cbioportal)

CASE_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,DNA_INPUT,MATCHED_STATUS,METASTATIC_SITE,ONCOTREE_CODE,OS_MONTHS,OS_STATUS,PRIMARY_SITE,SAMPLE_CLASS,SAMPLE_COLLECTION_SOURCE,SAMPLE_COVERAGE,SAMPLE_TYPE,SEX,SMOKING_HISTORY,SPECIMEN_PRESERVATION_TYPE,SPECIMEN_TYPE,TUMOR_PURITY,VITAL_STATUS
P-0005947-T01-IM5,Bladder Cancer,Upper Tract Urothelial Carcinoma,250,Matched,,UTUC,19.00,LIVING,Upper Tract,Tumor,In-House,648,Primary,Female,Unknown,DNA,Resection,80,ALIVE
P-0009180-T01-IM5,Melanoma,Cutaneous Melanoma,250,Matched,"Lymph Node, Regional",SKCM,NA,DECEASED,Skin,Tumor,In-House,750,Metastasis,Female,Never,FFPE,Resection,10,DECEASED
P-0008840-T01-IM5,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,250,Matched,Lymph Node,LUAD,1.78,LIVING,Lung,Tumor,In-House,978,Metastasis,Female,Prev/Curr Smoker,FFPE,Resection,30,ALIVE
P-0009304-T01-IM5,Glioma,Glioblastoma Multiforme,250,Matched,,GBM,1.22,LIVING,Brain,Tumor,Outside,1124,Primary,Female,Unknown,FFPE,Biopsy,50,ALIVE
P-0002010-T02-IM5,Ovarian Cancer,Clear Cell Ovarian Cancer,250,Matched,Liver,CCOV,6.28,DECEASED,Ovary,Tumor,In-House,862,Metastasis,Female,Never,FFPE,Resection,70,DECEASED
P-0004434-T01-IM5,Breast Cancer,Breast Invasive Lobular Carcinoma,250,Matched,Ipsilateral Breast,ILC,17.85,LIVING,Breast,Tumor,Outside,718,Metastasis,Female,Prev/Curr Smoker,DNA,Biopsy,50,ALIVE


In [35]:
get_table(unique(impact$Tumor_Sample_Barcode) %in% keys$Tumor_Sample_Barcode)
get_table(unique(impact$Tumor_Sample_Barcode) %in% cbioportal$CASE_ID)

unmatched_keys <- impact[! impact$Tumor_Sample_Barcode %in% keys$Tumor_Sample_Barcode,]
unmatched_cbio <- impact[! impact$Tumor_Sample_Barcode %in% cbioportal$CASE_ID,]

get_table(unique(unmatched_keys$Tumor_Sample_Barcode) %in% cbioportal$CASE_ID)

values,count,freq
TRUE,20040,87.2%
FALSE,2950,12.8%
-- total --,22990,100%


values,count,freq
FALSE,16283,70.8%
TRUE,6707,29.2%
-- total --,22990,100%


values,count,freq
FALSE,2064,70%
TRUE,886,30%
-- total --,2950,100%
